In [ ]:
import numpy as np
import pandas as pd
import sklearn
import time
from data import *

In [5]:
seedResults = createSeedResultsDF(seeds, results)

In [ ]:
nrow = seedResults.shape[0]
seedResults['WExpected'] = seedResults.WTeamID

start = time.time()

for i in range(nrow):
    if seedResults.loc[i, 'WNumSeed'] >= seedResults.loc[i, 'LNumSeed']:
        seedResults.loc[i, 'WExpected'] = seedResults.loc[i, 'LTeamID']

end = time.time()

print(end - start)

seedResults['correctGuess'] = seedResults.WExpected == seedResults.WTeamID

In [ ]:
accuracy = seedResults.correctGuess.sum() / nrow
print("Picking highest seed strategy accuracy: ", accuracy)

In [6]:
seedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSection,WNumSeed,LSection,LNumSeed
0,1985,136,1116,63,1234,54,N,0,X,9.0,X,8.0
1,1985,136,1120,59,1345,58,N,0,Z,11.0,Z,6.0
2,1985,138,1120,66,1242,64,N,0,Z,11.0,Z,3.0
3,1985,136,1207,68,1250,43,N,0,W,1.0,W,16.0
4,1985,138,1207,63,1396,46,N,0,W,1.0,W,8.0
